In [1]:
import os
import sys
import warnings
import pandas as pd
import numpy as np

warnings.filterwarnings("ignore", category=UserWarning, module="pandas")

PROJECT_ROOT = r"D:\demand_forecasting_system"
SRC_DIR = os.path.join(PROJECT_ROOT, "src")
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

from tasks.extract_mssql import fetch_table_data
results = fetch_table_data("blinkit_weather_data")  # adjust workers for speed

D:\demand_forecasting_system\src\tasks\extract_mssql.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
2025-10-21 21:09:00,565 | INFO | data_pipeline | Fetched 186600 rows from table 'blinkit_weather_data' (DB)


In [5]:
df_weather = pd.DataFrame(results)
print(df_weather.head())
print(df_weather.info())
print(df_weather.shape)

    area        date  temperature  precipitation
0  Adoni  2023-03-16          7.6            7.5
1  Adoni  2023-03-17          9.6            0.0
2  Adoni  2023-03-18         12.2            0.0
3  Adoni  2023-03-19         15.0            0.0
4  Adoni  2023-03-20         17.0            0.5
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186600 entries, 0 to 186599
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   area           186600 non-null  object 
 1   date           186600 non-null  object 
 2   temperature    186600 non-null  float64
 3   precipitation  186600 non-null  float64
dtypes: float64(2), object(2)
memory usage: 5.7+ MB
None
(186600, 4)


Convert Dates Properly

In [6]:
df_weather["date"] = pd.to_datetime(df_weather["date"], errors="coerce")


Check for Missing or Invalid Values

In [7]:
print(df_weather.isnull().sum())


area             0
date             0
temperature      0
precipitation    0
dtype: int64


In [13]:
df_weather["temperature"] = df_weather["temperature"].fillna(df_weather["temperature"].mean())
df_weather["precipitation"] = df_weather["precipitation"].fillna(0)



Ensure Data Consistency by Area

In [14]:
df_weather.drop_duplicates(subset=["area", "date"], inplace=True)


In [15]:
df_weather.sort_values(by=["area", "date"], inplace=True)
df_weather.reset_index(drop=True, inplace=True)


In [16]:
print(df_weather.dtypes)
print(df_weather.shape)


area                     object
date             datetime64[ns]
temperature             float64
precipitation           float64
dtype: object
(186600, 4)


In [17]:
import os
save_path = r"D:\demand_forecasting_system\data\processed\blinkit_weather_clean.csv"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
df_weather.to_csv(save_path, index=False)
print(f" Weather data cleaned and saved to: {save_path}")


 Weather data cleaned and saved to: D:\demand_forecasting_system\data\processed\blinkit_weather_clean.csv
